 # Libraries Installation

### OS Issues

* EPANET Simulator under wntr doesn't work in MacOS, Linux but does in Windows
* Windows has (.dll) precompiled but MacOS needs it to be compiled (.dylib)
* Working on compiling .dylib and .so for MacOS and Linux

In [1]:
# MacOS specific .dylib 
from pathlib import Path
import os
import platform

system_os = platform.system()

# Get current script directory
base_path = Path().resolve()

if system_os == "Darwin":  # macOS
    epanet_lib_path = base_path / "lib" / "libepanet.dylib"
    os.environ["EPANET_PATH"] = str(epanet_lib_path)

elif system_os == "Linux":
    epanet_lib_path = base_path / "lib" / "libepanet.so"
    os.environ["EPANET_PATH"] = str(epanet_lib_path)

elif system_os == "Windows":
    print("Windows detected. No need to set EPANET_PATH.")

else:
    raise EnvironmentError(f"Unsupported OS: {system_os}")



In [2]:
# Upgrade pip and install packages from requirements.txt (cross-platform safe)
%pip install --upgrade pip
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Genetic Algorithm

## Setting up Manual Functions

### Importing Libraries

In [3]:
import wntr
import numpy as np
from datetime import timedelta
import pandas as pd
import random
import copy
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

### Auxiliary Functions

In [4]:
# --- RESHAPE INDIVIDUAL TO SCHEDULE ---

def reshape_individual(individual, wntk):

    # Reshape the individual to a 2D array
    schedule = np.array(individual).reshape(wntk.describe(level=1)['Links']['Pumps'], (int(wntk.options.time.duration / wntk.options.time.hydraulic_timestep) + 1)).tolist()

    return schedule

In [5]:
# --- SCHEDULE ADDITION ---

def add_schedule(schedule, wntk):
    for i in range(wntk.describe(level=1)['Links']['Pumps']):
        for j in range(int(wntk.options.time.duration / wntk.options.time.hydraulic_timestep) + 1):
            pump = wntk.get_link(wntk.pump_name_list[i])
            condition = wntr.network.controls.SimTimeCondition(wntk, '=', str(timedelta(hours=j)))
            action = wntr.network.controls.ControlAction(pump, 'status', schedule[i][j])
            control = wntr.network.controls.Control(condition, action, name=f'Control_pump{i}_time{j}')
            wntk.add_control(f"Control Pump ID : {i}, Hour : {j}", control)

In [22]:
# --- ADDED SCHEDULE REMOVAL {EXCLUSIVELY THE ONES FROM ADD SCHEDULE}---

def remove_schedule(wntk):
    for i in range(wntk.describe(level=1)['Links']['Pumps']):
        for j in range(int(wntk.options.time.duration / wntk.options.time.hydraulic_timestep) + 1):
            wntk.remove_control(f"Control Pump ID : {i}, Hour : {j}")

In [28]:
# --- SWTICH COUNT FUNCTION ---

def count_switches(individual):
    """
    Counts the number of ON/OFF switches per pump for a single individual.

    Args:
        individual: list of lists; each inner list is a binary schedule (0 or 1) over time for one pump.

    Returns:
        List of integers: number of switches per pump.
    """
    switch_counts = []
    
    for schedule in individual:
        count = sum(schedule[i] != schedule[i - 1] for i in range(1, len(schedule)))
        switch_counts.append(count)
    
    return switch_counts



## Setting up DEAP framework

### Setting up Libraries

In [29]:
# Necessary Libraries for DEAP and Torch
from deap import base, tools, creator, algorithms

# --- CONFIGURATION ---
wn = wntr.network.WaterNetworkModel('Network Files/Net3.inp')
NUM_PUMPS = wn.describe(level=1)['Links']['Pumps']
NUM_TIMESTAMPS = (int(wn.options.time.duration / wn.options.time.hydraulic_timestep) + 1)
CHROM_SIZE = NUM_PUMPS * NUM_TIMESTAMPS  # No of Pumps * No of Timestamps

### Evaluation Function

In [30]:
# --- EVALUATION FUNCTION ---

def evaluation(individual, wntk = wn):

    wntk_copy = copy.deepcopy(wntk)

    # Reshape the individual to a schedule
    schedule = reshape_individual(individual, wntk_copy)

    # Get Total Number of Pump Switches
    switches_count = count_switches(schedule)

    # Add the schedule to the WNTR model
    add_schedule(schedule, wntk_copy)

    # Run the simulation
    sim = wntr.sim.WNTRSimulator(wntk_copy)
    results = sim.run_sim()

    # Reset the simulation
    remove_schedule(wntk_copy)
    wntk_copy.reset_initial_values()
    
    # --- Objectives ---

    # === Objective 1 - Total Cost of pump Operation ===

    gamma = 9.81
    timestep = wntk_copy.options.time.hydraulic_timestep
    duration_hr = timestep / 3600
    electricity_prices =[0.0244]*7+[0.1194]*15+[0.0244]*3
    pump_efficiency = 0.8

    total_pump_cost = 0
    num_price_steps = len(electricity_prices)
    max_power = 0

    for pump_name in wntk_copy.pump_name_list:
        pump = wntk_copy.get_link(pump_name)
        curve = wntk_copy.get_curve(pump.pump_curve_name)
        points = np.array(curve.points)
        flows = points[:, 0]
        heads = points[:, 1]

        pump_flows = results.link['flowrate'][pump_name].values

        for t, flow in enumerate(pump_flows[:num_price_steps]):
            if flow <= 0:
                continue

            head = np.interp(flow, flows, heads)
            power_kw = (gamma * head * flow) / (pump_efficiency)
            max_power = max(max_power, power_kw)

            cost = power_kw * electricity_prices[t] * duration_hr
            total_pump_cost += cost

    # === Objective 2 - Pump switch penalization ===
    cm = 5
    switch_penalty_cost = cm * sum(switches_count)

    # === Objective 3 - Cost of Demand Charges ===
    cd = 0.5
    demand_charge_cost = cd * max_power

    # --- Constraints ---

    constraint_penalty = 0

    # === Number of pump switches ===
    if any(switch > 5 for switch in switches_count):
        constraint_penalty += 1000000

    # === Tank Head constraints ===
    for tank_name in wntk_copy.tank_name_list:
        tank = wntk_copy.get_node(tank_name)
        min_level = tank.min_level
        max_level = tank.max_level
        tank_levels = results.node['head'][tank_name].values

        for t, level in enumerate(tank_levels):
            if level < min_level or level > max_level:
                constraint_penalty += 1000000

    # === Combined water volume constraints ===
    D_min = -10  # Minimum percentage difference (e.g., -5%)
    D_max = 10   # Maximum percentage difference (e.g., 5%)

    # Calculate Vs (sum of tank volumes at the start of the schedule)
    Vs = sum([wntk_copy.get_node(tank_name).init_level for tank_name in wntk.tank_name_list])

    # Calculate Ve (sum of tank volumes at the end of the schedule)
    Ve = sum([results.node['head'][tank_name].iloc[-1] for tank_name in wntk.tank_name_list])

    # Calculate percentage difference
    if Ve != 0:  # Avoid division by zero
        D = ((Vs - Ve) / Ve) * 100
        if D < D_min or D > D_max:
            constraint_penalty += 1000000  # Add a large penalty if the constraint is violated


    return total_pump_cost + switch_penalty_cost + demand_charge_cost + constraint_penalty,

In [31]:
# --- CREATE TYPES ---

# Assigning Mult-Class Fitness Weigths
creator.create("FitnessMulti", base.Fitness, weights=(-1.0,))

# Defining Individual
creator.create("Individual", list, fitness = creator.FitnessMulti)

toolbox = base.Toolbox()

# --- CREATE TOOLS ---

# Define Binary Genes
toolbox.register("attr_bool", random.randint, 0, 1)

# Define Individual (List of Binary Values)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=CHROM_SIZE)

# Define Random Population
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register the Fitness function
toolbox.register("evaluate", evaluation)

# Define Selection Methodology
toolbox.register("select", tools.selTournament, tournsize = 3)

# Define Crossover Methodology
toolbox.register("mate", tools.cxTwoPoint)

# Define Mutation Methodology
toolbox.register("mutate", tools.mutFlipBit, indpb = 0.1)

# Set Hall of Fame
hof = tools.HallOfFame(10)

In [ ]:
import os
from datetime import datetime

# --- GENETIC ALGO LOOP WITH RESULTS SAVING ---
def run_ga(pop_size=40, generations=50, cx_prob=0.7, mut_prob=0.3):
    # Create population
    population = toolbox.population(n=pop_size)

    # Objective Statistics
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # Run the Genetic Algorithm
    population, logbook = algorithms.eaSimple(
        population=population,
        toolbox=toolbox,
        cxpb=cx_prob,
        mutpb=mut_prob,
        ngen=generations,
        stats=stats,
        halloffame=hof,
        verbose=True,
    )

    # Get the best individual
    best_ind = hof[0]
    best_fitness = best_ind.fitness.values

    # Generate a timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    # Create a results directory if it doesn't exist
    results_dir = "Run_Results"
    os.makedirs(results_dir, exist_ok=True)

    # Write results to a file
    results_file = os.path.join(results_dir, f"GA_Run_{timestamp}.txt")
    with open(results_file, "w") as f:
        f.write(f"Timestamp: {timestamp}\n")
        f.write(f"Population Size: {pop_size}\n")
        f.write(f"Generations: {generations}\n")
        f.write(f"Crossover Probability: {cx_prob}\n")
        f.write(f"Mutation Probability: {mut_prob}\n\n")
        f.write(f"Best Individual: {best_ind}\n")
        f.write(f"Fitness: {best_fitness}\n\n")
        f.write("Logbook:\n")
        for record in logbook:
            f.write(f"{record}\n")

    print(f"Results saved to {results_file}")

    return best_ind, logbook, population, hof
'''
# Run the GA with logging
if __name__ == "__main__":
    best_sol, logbook, population, hof = run_ga()
'''

In [ ]:
best_sol, logbook, population, hof = run_ga(1000, 50)

# Test Runs

# Solution Analysis

In [ ]:
# Enter the individual value to Analyze
analysis_individual = best_sol
print(best_sol)

## Analysis Functions

In [ ]:
# --- COST ANALYSIS FUNCTION ---

def cost_analysis(analysis_individual, cost_pattern = np.array([0.7]*6 + [1.2]*12 + [1]*7).T, wntk=wn): # Cost pattern needs to be changed as per Time requirement (size must be equal to the number of time steps + 1)
    
    wntk_copy = copy.deepcopy(wntk)

    # Reshape the individual to a schedule
    schedule = reshape_individual(analysis_individual, wntk_copy)

    # Add the schedule to the WNTR model
    add_schedule(schedule, wntk_copy)
    
    # Get Cost Objective - NOTE !!! Price pattern is defined in cost objective function and can be changed inside the cost objective function if necessary
    cost_df = cost_objective(wntk_copy)

    # Analysis Plots


    # Reset the simulation
    remove_schedule(wntk_copy)
    wntk_copy.reset_initial_values()

In [ ]:
# --- DEMAND ANALYSIS FUNCTION ---

def demand_analysis(analysis_individual, wntk=wn):
    wntk_copy = copy.deepcopy(wntk)

    # Reshape individual into a schedule and add it to the network
    schedule = reshape_individual(analysis_individual, wntk_copy)
    add_schedule(schedule, wntk_copy)

    # Get the demand objective (WSA)
    wsa_df = demand_objective(wntk_copy)

    # Check the minimum and maximum WSA values across all time steps
    min_val = wsa_df.min().min()
    max_val = wsa_df.max().max()

    # Adjust node range for visualization (based on the actual min and max values)
    node_range = [max(0, min_val), min(1.0, max_val)]

    # Select timesteps to visualize (e.g., 9 evenly spaced)
    num_plots = 9
    time_indices = np.linspace(0, len(wsa_df.index)-1, num_plots, dtype=int)
    selected_times = wsa_df.index[time_indices]

    # Set up subplot grid
    ncols = 3
    nrows = int(np.ceil(num_plots / ncols))
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 10))
    axes = axes.flatten()

    # Create a colormap with NaN handling
    cmap = plt.cm.turbo  # Default colormap

    # Set up a custom colormap that handles NaN as a specific color
    colors = cmap(np.linspace(0, 1, 256))
    colors[0] = [0.8, 0.8, 0.8, 1]  # Set NaN to a light gray (or use a custom color)
    custom_cmap = mcolors.ListedColormap(colors)

    # Plot WSA for selected timesteps
    for i, time in enumerate(selected_times):
        ax = axes[i]
        wsa = wsa_df.loc[time, :]

        # Explicitly handle NaN by filling them with a specific color (gray)
        # Create a mask for NaN values
        wsa_masked = wsa.copy()
        wsa_masked = wsa_masked.fillna(np.nan)  # Ensure NaNs are not ignored

        # Plot the network with NaN treated as gray
        wntr.graphics.plot_network(
            wn,
            node_attribute=wsa_masked,
            node_size=20,
            node_range=node_range,
            node_cmap=custom_cmap,
            title=f'Time: {time//3600} hrs',
            ax=ax,
            show_plot=False
        )

    # Turn off unused axes
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()

    # Reset the simulation
    remove_schedule(wntk_copy)
    wntk_copy.reset_initial_values()


## Cost Analysis

In [ ]:
cost_analysis(analysis_individual)

## Demand Analysis

In [ ]:
demand_analysis(analysis_individual)